# 爬取豆瓣图书

In [140]:
import re
import requests
from lxml import etree
from bs4 import BeautifulSoup

import winsound

In [161]:
headers = {
    'User-Agent':      'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36',
    'Accept':          'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'zh-CN,zh;q=0.8,en-US;q=0.6,en;q=0.4',
    'Host': 'book.douban.com',
    'Cookie':'bid=NMutUAiHMz8; ll="118240"; gr_user_id=ca38b48e-47f2-4aec-b3ec-bea004b2fe3f; ct=y; __utmt_douban=1; __utmt=1; _pk_ref.100001.3ac3=%5B%22%22%2C%22%22%2C1472234608%2C%22https%3A%2F%2Fwww.baidu.com%2Flink%3Furl%3D5M8V2XhGLAyReyVy2c_w9S8Ep8tJhqAwgw3p726o9YMXLAs-WAS1NTdLYYM2YvyTwjGlzacjFXUoDBFNkBc-ra%26wd%3D%26eqid%3Ddb593da2000a01c30000000457bb232a%22%5D; ap=1; viewed="25985021_1449351_1082334_25862578_25753386_3004255_4031906_5355285_3117898_3652388"; _vwo_uuid_v2=148A5B7EFE845D63192E29CFC21E4E90|bf59b045734642da66b481151f99965e; gr_session_id_22c937bbd8ebd703f2d8e9445f7dfd03=7b4f4a1d-87a4-478c-a09f-a6898edf37bb; gr_cs1_7b4f4a1d-87a4-478c-a09f-a6898edf37bb=user_id%3A0; _pk_id.100001.3ac3=0b7e5ef3290af755.1468862827.12.1472235051.1472228838.; _pk_ses.100001.3ac3=*; __utma=30149280.1221162141.1468324283.1472228767.1472234607.26; __utmb=30149280.2.10.1472234607; __utmc=30149280; __utmz=30149280.1471882039.22.19.utmcsr=baidu|utmccn=(organic)|utmcmd=organic; __utma=81379588.22471417.1468862827.1472228767.1472234607.13; __utmb=81379588.2.10.1472234607; __utmc=81379588; __utmz=81379588.1471882039.9.7.utmcsr=baidu|utmccn=(organic)|utmcmd=organic',
    
}
url = 'https://book.douban.com/subject/25985021/'

In [146]:
soup.select("h1 span")[0].text

'呐喊'

In [162]:
req = requests.get(url,headers = headers)

In [163]:
req.status_code

200

In [153]:
c
# 获取书籍评分、评论人数
id = url.split('/')[4]
title = soup.select("h1 span")[0].text

IndexError: list index out of range

In [152]:
soup

<html>
<head><title>403 Forbidden</title></head>
<body bgcolor="white">
<center><h1>403 Forbidden</h1></center>
<hr/><center>nginx</center>
</body>
</html>
<!-- a padding to disable MSIE and Chrome friendly error page -->

In [164]:
def get_book_info(url, headers= {}):
    """
    args:
    url: 为书籍的链接，例如：'https://book.douban.com/subject/5989373/'

    returns:
    一个字典，包括书籍的元信息。
    """
    if not headers:
        headers = {
        'User-Agent':      'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36',
        'Accept':          'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'zh-CN,zh;q=0.8,en-US;q=0.6,en;q=0.4'
        }
    html = requests.get(url, headers=headers).text

    soup = BeautifulSoup(html, 'lxml')
    # 获取书籍评分、评论人数
    id = url.split('/')[4]
    title = soup.select("h1 span")[0].text
    tags = [tag.text for tag in soup.select("#db-tags-section a.tag")]
    rate = soup.select(
        '#interest_sectl  div.rating_self.clearfix > strong')[0].text
    rate_num = soup.select(
        '#interest_sectl  div.rating_self.clearfix  div.rating_sum  a > span')[0].text

    # 获取书籍信息
    children = list(soup.select('#info')[0].children)
    br = BeautifulSoup('<br/>', 'lxml').select('br')[0]

    # 找到 <br/> 的位置
    indexs = [i for i, child in enumerate(children) if child == br]

    childs = [children[:indexs[0]]]
    count = len(indexs)
    for i in range(count - 1):
        childs.append(children[indexs[i]:indexs[i + 1]])
    childs.append(children[indexs[-1]:])

#     child_n = [['书名',title],['评分', rate], ['评论人数', rate_num]]
    child_n = {'编号':id,'标题': title, '评分': rate, '评论人数': rate_num ,'标签': tags}
    for child in childs:
        string = ''.join([ch if ch.name == None else ch.text for ch in child])
        row = re.sub('\s+', '', string).split(':', 1)
        if len(row) == 2:
            child_n[row[0]] = row[1]
#             child_n.append(row)

    return child_n

In [3]:
tags = []
with open('tags.txt', 'r', encoding='utf-8') as file:
    tags = [line.strip() for line in file]

In [7]:
print(*tags,sep=", ")

文化, 小说, 中国文学, 当代文学, 古典文学, 外国文学, 外国名著, 推理, 科幻, 历史, 人物传记, 心理学, 哲学, 艺术, 教育, 儿童, 中学, 初中, 高中, 励志, 青春, 校园, 科学, 科普, 儿子读, 绘本, 经济, 管理, 健康


In [172]:
def douban_tag_ranks(tag, headers = {}):
    url = 'https://book.douban.com/tag/' + tag + '?type=S'
    r = requests.get(url,headers = headers)
    soup = BeautifulSoup(r.text, 'lxml')
    a_tags = soup.select('#subject_list > ul > li > div.info > h2 > a')

#     for i, a in enumerate(a_tags):
#         href = a.get('href')
#         title = re.sub('\s+', '', a.text)
#         print(href, title)
    return [(a.get('href'),re.sub('\s+', '', a.text)) for i, a in enumerate(a_tags)]

In [75]:
def get_book_to_xlsx(tag):
    row_title = ['编号','ISBN', '标题','副标题','作者','评分','评价人数','出版社','原作者',
                 '译者','出版年','页数','定价','装帧','丛书']
    book_tags = douban_tag_ranks(tag)
    books = []
    for href,title in book_tags:
        headers['Referer'] = 'https://book.douban.com/tag/'+ tag,
        book_infos = get_book_info(href ,headers = headers)
        keys = book_infos.keys()
#         for meta in row_title:
#             if meta in keys():
#                 book_infos[meta]
        infos = [book_infos[meta] if meta in keys else None for meta in row_title]
        books.append([tag] + infos)
    return books

In [55]:
get_book_to_xlsx('文化')

'5989373'

In [77]:
import xlwings as xw
from pymongo import MongoClient

In [78]:
client = MongoClient('mongodb://10.57.0.92:27017')
db = client['maple_pds']
print('connect maple_pds')

connect maple_pds


In [103]:
def get_book_to_db(tag):
    book_tags = douban_tag_ranks(tag)
    books = []
    headers = {
        'User-Agent':      'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36',
        'Accept':          'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'zh-CN,zh;q=0.8,en-US;q=0.6,en;q=0.4'
    }
    for href,title in book_tags:
#         headers['Referer'] = 'https://book.douban.com/tag/'+ tag,
        book_infos = get_book_info(href, headers = headers)
        book_infos['tag'] = tag
        book_infos['_id'] = {book_infos['ISBN']}
        books.append(book_infos)
    return books

In [111]:
db.books.delete_many({})

In [116]:
_.deleted_count

20

In [134]:
for i in db.books.find({}):
    print(i)

In [143]:
get_book_info('https://book.douban.com/subject/1449351/')

IndexError: list index out of range

In [165]:
import time

In [173]:
douban_tag_ranks('文化',headers = headers)

[('https://book.douban.com/subject/25985021/', '人类简史:从动物到上帝'),
 ('https://book.douban.com/subject/4238362/', '送你一颗子弹'),
 ('https://book.douban.com/subject/1027191/', '历史深处的忧虑:近距离看美国之一'),
 ('https://book.douban.com/subject/5375620/', '失控:全人类的最终命运和结局'),
 ('https://book.douban.com/subject/1022060/', '狼图腾'),
 ('https://book.douban.com/subject/3162991/', '艺术的故事'),
 ('https://book.douban.com/subject/20440644/', '文学回忆录（全2册）:1989—1994'),
 ('https://book.douban.com/subject/1001885/', '带一本书去巴黎'),
 ('https://book.douban.com/subject/1062193/', '娱乐至死'),
 ('https://book.douban.com/subject/1014825/', '我也有一个梦想:近距离看美国之三'),
 ('https://book.douban.com/subject/4124727/', '孤独六讲'),
 ('https://book.douban.com/subject/1056315/', '总统是靠不住的:近距离看美国之二'),
 ('https://book.douban.com/subject/3344676/', '常识'),
 ('https://book.douban.com/subject/1426971/', '野火集:二十年纪念版'),
 ('https://book.douban.com/subject/1795079/', '乡土中国'),
 ('https://book.douban.com/subject/1022238/', '菊与刀:日本文化的类型'),
 ('https://book.douban.com/subjec

In [174]:
for tag in tags:
    book_tags = douban_tag_ranks(tag, headers = headers)
    i = 0
    for href,title in book_tags:
#         headers['Referer'] = 'https://book.douban.com/tag/'+ tag,
        try:
            book_infos = get_book_info(href, headers = headers)
            book_infos['tag'] = tag
            book_infos['title'] = title
            book_infos['_id'] = book_infos['编号']
        except:
            print('x:', tag, title)
        else:
            db.books.replace_one({'_id':book_infos['编号']}, book_infos, upsert=True)
        print(i, tag, title)
        time.sleep(1)
        i += 1
winsound.PlaySound('alert', winsound.SND_ASYNC)

0 文化 人类简史:从动物到上帝
1 文化 送你一颗子弹
2 文化 历史深处的忧虑:近距离看美国之一
3 文化 失控:全人类的最终命运和结局
4 文化 狼图腾
5 文化 艺术的故事
6 文化 文学回忆录（全2册）:1989—1994
7 文化 带一本书去巴黎
8 文化 娱乐至死
9 文化 我也有一个梦想:近距离看美国之三
10 文化 孤独六讲
11 文化 总统是靠不住的:近距离看美国之二
12 文化 常识
13 文化 野火集:二十年纪念版
14 文化 乡土中国
15 文化 菊与刀:日本文化的类型
16 文化 文化苦旅
17 文化 西班牙旅行笔记
18 文化 美的历程
0 小说 追风筝的人
1 小说 围城
2 小说 活着
3 小说 平凡的世界（全三部）
4 小说 百年孤独
5 小说 月亮和六便士
6 小说 解忧杂货店
7 小说 霍乱时期的爱情
8 小说 哈利·波特与魔法石
9 小说 三国演义（全二册）
10 小说 肖申克的救赎
11 小说 哈利·波特与火焰杯
12 小说 哈利·波特与阿兹卡班的囚徒
13 小说 倾城之恋
14 小说 飘（上下）
15 小说 哈利·波特与密室
16 小说 不能承受的生命之轻
17 小说 黄金时代:时代三部曲
18 小说 哈利·波特与死亡圣器
19 小说 达·芬奇密码
0 中国文学 围城
1 中国文学 活着
2 中国文学 平凡的世界（全三部）
3 中国文学 沉默的大多数:王小波杂文随笔全编
4 中国文学 倾城之恋
5 中国文学 送你一颗子弹
6 中国文学 黄金时代:时代三部曲
7 中国文学 文学回忆录（全2册）:1989—1994
8 中国文学 边城
9 中国文学 许三观卖血记
10 中国文学 明朝那些事儿（1-9）:限量版
11 中国文学 一只特立独行的猪
12 中国文学 狼图腾
13 中国文学 穆斯林的葬礼
14 中国文学 呐喊
15 中国文学 我的精神家园:王小波杂文自选集
16 中国文学 1988：我想和这个世界谈谈
17 中国文学 白鹿原
18 中国文学 兄弟（上）
19 中国文学 诗经
0 当代文学 夹边沟记事
1 当代文学 尘埃落定
2 当代文学 城门开
3 当代文学 红拂夜奔
4 当代文学 一句顶一万句
5 当代文学 小姨多鹤
6 当代文学 棋王
7 当代文学 动物凶猛
8 当代文学 妻妾成群
9 当代文学 额尔古纳河右岸


In [205]:
book = xw.Book('data/books.xlsx')

In [206]:
sheet = book.sheets[0]

In [207]:
a1 = sheet.range('A1')
a1.value = row_title

In [209]:
a1.offset(0,4).value

'书名'

In [196]:
cur = db.books.find({})

In [212]:
list(db.books.find({'_id': '2295163'}))[0]

{'ISBN': '9787020063659',
 '_id': '2295163',
 'tag': '中学',
 'title': '哈利·波特与死亡圣器',
 '丛书': '哈利·波特',
 '作者': '[英]J.K.罗琳',
 '出版年': '2007-10',
 '出版社': '人民文学出版社',
 '原作名': 'HarryPotterandtheDeathlyHallows',
 '定价': '66.00元',
 '标签': ['哈利波特', 'J.K.罗琳', '魔幻', '小说', '英国', 'HarryPotter', '哈利·波特', '外国文学'],
 '标题': '哈利·波特与死亡圣器',
 '编号': '2295163',
 '装帧': '平装',
 '评分': ' 8.9 ',
 '评论人数': '52073',
 '译者': '马爱农/马爱新',
 '页数': '574'}

In [202]:
row_title = ['序号','编号', 'ISBN','标签', '书名','作者','评分','评论人数','出版社',
                 '译者','出版年','页数','定价','装帧','丛书']
row_meta = ['_id', 'ISBN','tag', 'title','作者','评分','评论人数','出版社',
                 '译者','出版年','页数','定价','装帧','丛书']

In [215]:
cur = db.books.find({})
cell_A = sheet.range('A1')
cell_A.value = row_title
for i, c in enumerate(cur):
    keys = c.keys()
    row = [i+1]
    for meta in row_meta:
        if meta in keys:
            row.append(c[meta])
        else:
            row.append(None)
    cell_A = cell_A.offset(1)
    cell_A.value = row
    hyperlink = "https://book.douban.com/subject/"+ row[1] +"/"
    cell_A.offset(column_offset=4).add_hyperlink(hyperlink, row[4])

In [118]:
def get_book_to_xlsx(tag):
    row_title = ['编号', 'ISBN', '标题','副标题','作者','评分','评价人数','出版社','原作者',
                 '译者','出版年','页数','定价','装帧','丛书']
    book_tags = douban_tag_ranks(tag)
    books = []
    for href,title in book_tags:
        headers['Referer'] = 'https://book.douban.com/tag/'+ tag,
        book_infos = get_book_info(href ,headers = headers)
        keys = book_infos.keys()
#         for meta in row_title:
#             if meta in keys():
#                 book_infos[meta]
        infos = [book_infos[meta] if meta in keys else None for meta in row_title]
        books.append([tag] + infos)
    return books

{'ISBN': '9787508647357',
 '丛书': '“中信史学大师畅销经典”系列',
 '书名': '人类简史',
 '作者': '[以色列]尤瓦尔·赫拉利',
 '出版年': '2014-11-1',
 '出版社': '中信出版社',
 '副标题': '从动物到上帝',
 '原作名': 'Abriefhistoryofhumankind',
 '定价': 'CNY68.00',
 '装帧': '平装',
 '评分': ' 9.3 ',
 '评论人数': '11724',
 '译者': '林俊宏',
 '页数': '440'}